In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from scipy import sparse
import sys
from sklearn.utils import shuffle
import joblib
sys.path.append("../")
from helper_functions import *

In [ ]:
x_train = sparse.load_npz("Data/xtrain.npz")
x_test = sparse.load_npz("Data/xtest.npy.npz")
y_train = np.load("Data/ytrain.npy")
y_test = np.load("Data/ytest.npy")

In [ ]:
x_train = x_train.toarray()
x_test = x_test.toarray()
x = np.concatenate([x_train, x_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)
x = sparse.csr_matrix(x)

In [ ]:
x,y = shuffle(x,y, random_state=1)

In [ ]:
log_reg = LogisticRegression()
lgbm = LGBMClassifier()
gboost = GradientBoostingClassifier()
cat = CatBoostClassifier()

In [ ]:
models = {
    'LogisticRegression': log_reg,
    'LGBMClassifier': lgbm,
    'GradientBoostingClassifier': gboost,
    'CatBoostClassifier': cat
}

In [ ]:
BestParam_search(models, x, y)